In [1]:
!pip install --pre deepchem
import deepchem as dc
from deepchem.feat.mol_graphs import ConvMol

!pip install -U scikit-learn
from sklearn import metrics, multioutput, ensemble, linear_model

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import math
import datetime
import os
import pytest

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 5.1 MB/s 
     |████████████████████████████████| 36.8 MB 24 kB/s 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
def get_top_k_precision(y_true, y_pred, k=20):
  top_k_idx = np.argpartition(y_pred, -k, axis=0)[-k:]
  top_k_labels = y_pred[top_k_idx]
  top_k_precision = np.mean(top_k_labels)
  return top_k_precision


In [3]:
def negative_precision(y_true, y_pred):
  return metrics.precision_score(1 - y_true, 1 - y_pred)
  

In [9]:
#recall, precision, top 20 precision, negative precision
def get_multilabel_stats(model, X, y, k):
  prediction = model.predict(X)
  functions = [metrics.recall_score, metrics.precision_score, get_top_k_precision, negative_precision]
  result = np.zeros([len(functions), y.shape[1]])
  for i in range(len(functions)):
    for j in range(y.shape[1]):
      result[i,j] = functions[i](y[:,j], prediction[:,j])
  return result

In [15]:
#get fingerprints
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='ECFP', splitter='random')
train_dataset, valid_dataset, test_dataset = datasets

In [16]:
classifier = linear_model.LogisticRegression()
regression_model = multioutput.MultiOutputClassifier(classifier)
regression_model.fit(train_dataset.X, train_dataset.y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


MultiOutputClassifier(estimator=LogisticRegression())

In [17]:
#train set stats
get_multilabel_stats(regression_model, train_dataset.X, train_dataset.y, 20)

array([[0.572     , 0.6974359 , 0.58978583, 0.47058824, 0.33064516,
        0.53521127, 0.52597403, 0.37402597, 0.43661972, 0.37755102,
        0.54098361, 0.46264368],
       [0.94701987, 0.97142857, 0.8817734 , 0.96551724, 0.8436214 ,
        0.93251534, 0.97590361, 0.82758621, 0.93939394, 0.94871795,
        0.86462882, 0.94152047],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        ],
       [0.98249632, 0.99036577, 0.95749403, 0.97950553, 0.93107457,
        0.9783642 , 0.98818961, 0.91852603, 0.98053528, 0.97022938,
        0.94212883, 0.96930904]])

In [18]:
#valid set stats
get_multilabel_stats(regression_model, valid_dataset.X, valid_dataset.y, 20)

array([[0.4137931 , 0.42105263, 0.37662338, 0.11538462, 0.19      ,
        0.25      , 0.13333333, 0.20481928, 0.0952381 , 0.13157895,
        0.35051546, 0.19444444],
       [0.54545455, 0.53333333, 0.59183673, 0.375     , 0.51351351,
        0.45454545, 0.5       , 0.31481481, 0.22222222, 0.33333333,
        0.56666667, 0.38888889],
       [1.        , 0.75      , 1.        , 0.4       , 1.        ,
        1.        , 0.2       , 1.        , 0.45      , 0.75      ,
        1.        , 0.9       ],
       [0.97766097, 0.98567708, 0.9346049 , 0.97032258, 0.89142091,
        0.96057819, 0.98331194, 0.90946502, 0.9754522 , 0.95703125,
        0.91286307, 0.9620915 ]])

In [41]:
forest_model = ensemble.RandomForestClassifier(min_samples_split=10)
forest_model.fit(train_dataset.X, train_dataset.y)

RandomForestClassifier(min_samples_split=10)

In [42]:
get_multilabel_stats(forest_model, train_dataset.X, train_dataset.y, 20)

array([[0.476     , 0.53846154, 0.46952224, 0.2605042 , 0.27419355,
        0.33098592, 0.05844156, 0.30519481, 0.15023474, 0.10884354,
        0.49590164, 0.20114943],
       [0.89473684, 0.91304348, 0.9965035 , 0.92537313, 0.94972067,
        0.95918367, 1.        , 0.99156118, 1.        , 0.96969697,
        0.98641304, 0.98591549],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 0.45      , 1.        , 1.        , 1.        ,
        1.        , 1.        ],
       [0.97863318, 0.98536347, 0.94613583, 0.97159916, 0.92604766,
        0.96918586, 0.97681855, 0.91123279, 0.97095635, 0.95795217,
        0.9374152 , 0.95511061]])

In [43]:
get_multilabel_stats(forest_model, valid_dataset.X, valid_dataset.y, 20)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[0.37931034, 0.42105263, 0.06493506, 0.07692308, 0.12      ,
        0.15      , 0.        , 0.04819277, 0.        , 0.        ,
        0.09278351, 0.05555556],
       [0.6875    , 0.66666667, 0.83333333, 0.66666667, 0.8       ,
        1.        , 0.        , 0.8       , 0.        , 0.        ,
        0.75      , 1.        ],
       [0.8       , 0.6       , 0.3       , 0.15      , 0.75      ,
        0.3       , 0.        , 0.25      , 0.        , 0.05      ,
        0.6       , 0.1       ],
       [0.97653194, 0.98573281, 0.90733591, 0.96923077, 0.88541667,
        0.95624196, 0.98084291, 0.89845758, 0.97318008, 0.95140665,
        0.88586252, 0.95646607]])